# Probability 

This IPy notebook acts as supporting material for **Chapter 13 Quantifying Uncertainty**, **Chapter 14 Probabilistic Reasoning** and **Chapter 15 Probabilistic Reasoning over Time** of the book* Artificial Intelligence: A Modern Approach*. This notebook makes use of the implementations in probability.py module. Let us import everything from the probability module. It might be helpful to view the source of some of our implementations. Please refer to the Introductory IPy file for more details on how to do so.

In [ ]:
from probability import *

## Probability Distribution

Let us begin by specifying discrete probability distributions. The class **ProbDist** defines a discrete probability distribution. We name our random variable and then assign probabilities to the different values of the random variable. Assigning probabilities to the values works similar to that of using a dictionary with keys being the Value and we assign to it the probability. This is possible because of the magic methods **_ _getitem_ _**  and **_ _setitem_ _** which store the probabilities in the prob dict of the object. You can keep the source window open alongside while playing with the rest of the code to get a better understanding.

In [ ]:
%psource ProbDist

In [ ]:
p = ProbDist('Flip')
p['H'], p['T'] = 0.25, 0.75
p['T']

The first parameter of the constructor **varname** has a default value of '?'. So if the name is not passed it defaults to ?. The keyword argument **freqs** can be a dictionary of values of random variable:probability. These are then normalized such that the probability values sum upto 1 using the **normalize** method.

In [ ]:
p = ProbDist(freqs={'low': 125, 'medium': 375, 'high': 500})
p.varname


In [ ]:
(p['low'], p['medium'], p['high'])

Besides the **prob** and **varname** the object also separately keeps track of all the values of the distribution in a list called **values**. Every time a new value is assigned a probability it is appended to this list, This is done inside the **_ _setitem_ _** method.

In [ ]:
p.values

The distribution by default is not normalized if values are added incremently. We can still force normalization by invoking the **normalize** method.

In [ ]:
p = ProbDist('Y')
p['Cat'] = 50
p['Dog'] = 114
p['Mice'] = 64
(p['Cat'], p['Dog'], p['Mice'])

In [ ]:
p.normalize()
(p['Cat'], p['Dog'], p['Mice'])

It is also possible to display the approximate values upto decimals using the **show_approx** method.

In [ ]:
p.show_approx()